https://docs.aws.amazon.com/sagemaker/latest/dg/mxnet-example1.html

https://github.com/aws/sagemaker-python-sdk

http://sagemaker.readthedocs.io/en/latest/

In [1]:
import mxnet as mx
import numpy as np
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role
from io import BytesIO
import boto3

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


First, we define the Job that 

In [2]:
role = get_execution_role()

mnist_estimator = MXNet(entry_point='part2_sm_mnist.py',
                        py_version='py3',
                        role=role,
                        train_instance_count=1, 
                        train_instance_type='ml.p2.xlarge')

Now we can call the fit() method. This does a few things behind the scenes, but of note:
1. This fit() method's input argument is an S3 address (or address in a dictionary). The method then automatically downloads the file(s) onto the instance to a preset directory. Recall from part2 that these file locations are in the 'channel_input_dirs' argument going into the training script's train() method.
2. This method begins a Sagemaker Job, which you can find on the [Sagemaker Console](https://console.aws.amazon.com/sagemaker/home), under the "Jobs" tab on the left.
3. Once the job is complete, a Model will be created, which you can also find on the Sagemaker Console.

In [3]:
#mnist_estimator.fit("s3://jakechenawspublic/sample_data/mnist/train/")
mnist_estimator.fit(inputs={'images':'s3://jakechenawspublic/sample_data/mnist/train/images',
                            'labels':'s3://jakechenawspublic/sample_data/mnist/train/labels'})

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-148356438281
INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-py3-gpu-2018-01-14-00-01-31-025


.................................................................................
executing startup script (first run)
2018-01-14 00:08:09,661 INFO - root - running container entrypoint
2018-01-14 00:08:09,662 INFO - root - starting train task
2018-01-14 00:08:11,508 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'available_gpus': 1, 'output_data_dir': '/opt/ml/output/data/', 'user_script_name': 'part2_sm_mnist.py', 'hosts': ['algo-1'], 'available_cpus': 4, 'user_script_archive': 's3://sagemaker-us-east-1-148356438281/sagemaker-mxnet-py3-gpu-2018-01-14-00-01-31-025/source/sourcedir.tar.gz', 'enable_cloudwatch_metrics': False, 'code_dir': '/opt/ml/code', 'hyperparameters': {'sagemaker_container_log_level': 20, 'sagemaker_job_name': 'sagemaker-mxnet-py3-gpu-2018-01-14-00-01-31-025', 'sagemaker_program': 'part2_sm_mnist.py', 'sagemaker_region': 'us-east-1', 'sagemaker_submit_directory': 's3://sagemaker-us-east-1-148356438281/sagemaker-mxnet-py3-gpu-2018-01-14-00-01-31-025/sourc

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/sagemaker/session.py:630: DeprecationWarning: generator 'multi_stream_iter' raised StopIteration
  for idx, event in sagemaker.logs.multi_stream_iter(client, log_group, stream_names, positions):


2018-01-14 00:08:57,562 INFO - root - X_train.shape: (60000, 1, 28, 28)
2018-01-14 00:08:57,562 INFO - root - y_train.shape: (60000,)
[00:09:01] src/operator/././cudnn_algoreg-inl.h:106: Running performance tests to find the best convolution algorithm, this can take a while... (setting env variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
2018-01-14 00:09:03,981 INFO - root - Epoch[0] Train-accuracy=0.881117
2018-01-14 00:09:03,982 INFO - root - Epoch[0] Time cost=2.203
2018-01-14 00:09:06,173 INFO - root - Epoch[1] Train-accuracy=0.973967
2018-01-14 00:09:06,173 INFO - root - Epoch[1] Time cost=2.187
2018-01-14 00:09:08,364 INFO - root - Epoch[2] Train-accuracy=0.980733
2018-01-14 00:09:08,364 INFO - root - Epoch[2] Time cost=2.187
2018-01-14 00:09:10,553 INFO - root - Epoch[3] Train-accuracy=0.985050
2018-01-14 00:09:10,553 INFO - root - Epoch[3] Time cost=2.186
2018-01-14 00:09:12,744 INFO - root - Epoch[4] Train-accuracy=0.986983
2018-01-14 00:09:12,744 INFO - root - Epoch[4] 

Now we can deploy this model.

In [4]:
predictor = mnist_estimator.deploy(instance_type="ml.m4.xlarge",
                                   initial_instance_count=1,
                                   endpoint_name='tutorial-mnist-endpoint')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-py3-gpu-2018-01-14-00-01-31-025
INFO:sagemaker:Creating endpoint with name tutorial-mnist-endpoint


----------------------------------------------------------------------------------------------------------------------------!

Done! Let's now run a record through the predictor.

In [5]:
X_test = np.loadtxt('images_sm.csv', delimiter=',').reshape(-1, 1, 28, 28)

y_proba = np.array(predictor.predict(X_test))

y_proba.shape

(5, 10)

Looks good so far. 

In [6]:
mx.nd.argmax(mx.nd.array(y_proba), axis=1)


[ 7.  2.  1.  0.  4.]
<NDArray 5 @cpu(0)>

Once our predictor is deployed, any future application can take advantage of the prediction endpoint we created above. This allows us to close this development notebook and stop paying for the resources.

While we won't do that right now, we can demonstrate this by wiping the 'predictor' object from memory.

In [7]:
del(predictor)
predictor.predict(X_test)

NameError: name 'predictor' is not defined

Now, let's feed the records through the predictor we deployed from scratch. This can be done by invoking the endpoint using Boto3

In [8]:
s3 = boto3.client('s3')
sm = boto3.client('sagemaker')
sm_runtime = boto3.client('sagemaker-runtime')

First, let's load the new records from S3 into memory.

In [9]:
# Load the image records from S3 into local memory
resp = s3.get_object(
    Bucket='jakechenawspublic',
    Key='sample_data/mnist/test/images/images_sm.csv'
)

X_test = resp['Body'].read()   

Next, let's get the name of the prediction endpoint. If you forgot what this is, you can look it up on the [Sagemaker Console](https://console.aws.amazon.com/sagemaker/home) or by using Boto3 to list out what they are.

In [10]:
# Get prediction endpoint
resp = sm.list_endpoints()
for r in resp['Endpoints']:
    print(r['EndpointName'])

tutorial-mnist-endpoint


As shown above, our endpoint name is 'tutorial-mnist-predictor'. Let's now invoke this endpoint.

Sagemaker's endpoint API, by default, is set up to receive json or csv. For more info on how input data is processed by default, or on how to customize your own, see [this page](https://github.com/aws/sagemaker-python-sdk#input-processing).

We will be using Sagemaker's default CSV input. To do this, we modify both the "ContentType" and "Accept" parameters to 'text/csv', as shown below. For more information on these parameters, see the [Boto3 documentation for invoke_endpoint()](http://boto3.readthedocs.io/en/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)

In [12]:
# Send the dataset to the prediction endpoint
resp = sm_runtime.invoke_endpoint(
    EndpointName='tutorial-mnist-endpoint',
    Body=X_test,
    ContentType='text/csv', # -- These 2 are not required by the docs but very helpful.
    Accept='text/csv'       # /  Not sure why the values would be different.
)

In [13]:
# Load the response body back into an Numpy array for further processing
b = BytesIO(resp['Body'].read())
array = np.loadtxt(b, delimiter=',')
array.shape

(5, 10)

The shape looks good. Now let's see what the predicted numbers are:

In [14]:
mx.nd.argmax(mx.nd.array(array), axis=1)


[ 7.  2.  1.  0.  4.]
<NDArray 5 @cpu(0)>

Congratulations on reaching the end of this tutorial. To finish up, let's clean up the resources used here so we stop paying for the prediction endpoint. You can do that in the [Sagemaker Console](https://console.aws.amazon.com/sagemaker/home), or using the Boto3 API as demonstrated below.

In [ ]:
#resp = sm.delete_endpoint(EndpointName='tutorial-mnist-endpoint')
#print(resp['ResponseMetadata']['HTTPStatusCode'])